In [99]:
import requests

from config import settings
from pprint import pprint

In [30]:
def get_news_data(link: str):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0'
    }
    response = requests.get(link,  headers=headers)
    if response.status_code == 200:
        return response.json()
    raise 'Ошибка полученния данных с новостного сайта'

In [31]:
for type_link, links in settings.news_link.items():
    for link in links:
        if type_link == 'mun':
            content = get_news_data(link + '/api/site/matters')
            print(content)
            break
            get_actual_news(content)
        elif type_link == 'reg':
            pass
        else:
            raise f'Неожиданный тип новостных порталов: {type_link}'

{'matters': [{'id': 413020, 'title': '9 апреля 2024: какие сегодня праздники отмечают', 'is_promo': False, 'show_authors': False, 'published_at': '2024-04-09T09:30:00.000+11:00', 'noindex_text': None, 'platform_id': 12, 'lightning': False, 'registration_area': 'bashkortostan', 'is_partners_matter': False, 'is_exclusive': False, 'has_video': False, 'has_audio': False, 'has_gallery': False, 'images_count': 1, 'lead': 'Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.', 'kind': 'news_item', 'kind_path': '/news', 'image': {'author': None, 'source': '@kues1 / freepik.com', 'description': None, 'width': 800, 'height': 451, 'url': '/attachments/1e463007ba7e84e0adc017974e16d0d720688787/store/crop/0/24/800/451/800/0/0/0/85d204bcb33d68259a8918f7283eaa7da5c8c672a0b9480df766a2585196/85d204bcb33d68259a8918f7283eaa7da5c8c672a0b9480df766a2585196.jpeg.jpg', 'image_90': '/attachments/2b4b9943e2ea16c9fbaef9f489839f5868b9cfa5/store/crop/0/24/800/

### Реализация проверки даты 
путем прибавления к дате публикации +1 день и сравнение с текущим днем

In [54]:
from datetime import datetime, timedelta, timezone
print((datetime.fromisoformat(content['matters'][3]['published_at'])+ timedelta(days=1))> datetime.now(tz=timezone.utc))

False


In [51]:
datetime.now(tz=timezone.utc)

datetime.datetime(2024, 4, 9, 5, 48, 28, 644156, tzinfo=datetime.timezone.utc)

### Запись полученных данных в pydantic модель

In [79]:
import models.pydantic_mun_model
import importlib

importlib.reload(models.pydantic_mun_model)

from models.pydantic_mun_model import News

new_dict = {}

In [80]:
new_dict['path'] = content['matters'][0]['path']
new_dict['title']  = content['matters'][0]['title']
new_dict['published_at'] = content['matters'][0]['published_at']
new_dict['images_count'] = content['matters'][0]['images_count']
new_dict['lead'] = content['matters'][0]['lead']

new_news = News(**new_dict)


{'path': '/news/society/2024-04-09/9-aprelya-2024-kakie-segodnya-prazdniki-otmechayut-413020', 'title': '9 апреля 2024: какие сегодня праздники отмечают', 'published_at': '2024-04-09T09:30:00.000+11:00', 'images_count': 1, 'lead': 'Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.'}


In [100]:
# Создание объекта без выборки словаря
cur_news = News(**content['matters'][0])

### Получение статьи

In [95]:
# получение ссылки
article_link = link +'api/site/matters'+ new_news.path
# получение данных с сылки
article_data = requests.get(article_link)

if article_data.status_code == 200:
    article_json = article_data.json()
else:
    print(article_data)

https://sakhizdat.ru/api/site/matters/news/society/2024-04-09/9-aprelya-2024-kakie-segodnya-prazdniki-otmechayut-413020


In [117]:
importlib.reload(models.pydantic_mun_model)

from models.pydantic_mun_model import Article

cur_article = Article(**article_json)

In [104]:
cur_news.article = cur_article
print(cur_news)

title='9 апреля 2024: какие сегодня праздники отмечают' path='/news/society/2024-04-09/9-aprelya-2024-kakie-segodnya-prazdniki-otmechayut-413020' published_at=datetime.datetime(2024, 4, 9, 9, 30, tzinfo=TzInfo(+11:00)) images_count=1 lead='Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.' article=Article(title='9 апреля 2024: какие сегодня праздники отмечают', published_at=datetime.datetime(2024, 4, 9, 9, 30, tzinfo=TzInfo(+11:00)), lead='Газета «Южно-Сахалинск сегодня» рассказывает, какие праздники отмечаются в России и мире 9 апреля 2024 года.', tag=None, content=None)


In [110]:
response = requests.get('https://sakhizdat.ru/api/site/matters/articles/persona-1/2024-04-08/viktoriya-matvienko-vospitatel-dolzhen-nauchit-detey-obschatsya-druzhit-i-proschat-412754')
rez = response.json()
rez

Article(title='Виктория Матвиенко: воспитатель должен научить детей общаться, дружить и прощать', published_at=datetime.datetime(2024, 4, 8, 10, 28, tzinfo=TzInfo(+11:00)), lead='Жюри муниципального этапа конкурса «Педагог года - 2024» , который традиционно прошел в Южно-Сахалинске,  отметило ее в специальной номинации «За сохранение культурных традиций». ', tag=None, content=None, image=Image(author=None, source=None, description=None, width=631, height=356, image_90='/attachments/96955f96510ed9619513f54983f64f493af79eb6/store/crop/84/0/631/356/90/0/0/0/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee.jpeg.jpg', image_250='/attachments/084aca05b52a0bcf97a22637ad9bb8ef7caeba74/store/crop/84/0/631/356/250/0/0/0/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee/2c98659daa0974b9334c652fcbcea445cd0649c311232bd9a9cb130714ee.jpeg.jpg', image_800='/attachments/20a2fd873f1675ee41d35bce025f35adf9b58d79/store/cro

In [118]:
rez_article = Article(**rez)
rez_article

Article(title='Виктория Матвиенко: воспитатель должен научить детей общаться, дружить и прощать', published_at=datetime.datetime(2024, 4, 8, 10, 28, tzinfo=TzInfo(+11:00)), lead='Жюри муниципального этапа конкурса «Педагог года - 2024» , который традиционно прошел в Южно-Сахалинске,  отметило ее в специальной номинации «За сохранение культурных традиций». ', tags=[Tag(title='педагог', slug='pedagog', path='/tags/pedagog'), Tag(title='конкурс', slug='konkurs', path='/tags/konkurs'), Tag(title='детский сад', slug='detskiy-sad', path='/tags/detskiy-sad'), Tag(title='семья', slug='semya', path='/tags/semya'), Tag(title='образование', slug='obrazovanie', path='/tags/obrazovanie'), Tag(title='традиции', slug='traditsii', path='/tags/traditsii')], content_blocks=[Content(position=0, kind='common', text='<p><em>Общий стаж работы в сфере образования - 13 лет. За это время прошла профессиональную переподготовку и освоила методы, которые помогают увлечь полезным даже самых непоседливых. В 2018-м 

In [119]:
rez_article.tags

[Tag(title='педагог', slug='pedagog', path='/tags/pedagog'),
 Tag(title='конкурс', slug='konkurs', path='/tags/konkurs'),
 Tag(title='детский сад', slug='detskiy-sad', path='/tags/detskiy-sad'),
 Tag(title='семья', slug='semya', path='/tags/semya'),
 Tag(title='образование', slug='obrazovanie', path='/tags/obrazovanie'),
 Tag(title='традиции', slug='traditsii', path='/tags/traditsii')]

In [2]:
import requests
from mun_model import News, Article
link = 'https://sakhizdat.ru/'
# Получаем новости
response_news = requests.get('https://sakhizdat.ru/api/site/matters')
for news in response_news.json()['matters']:
    cur_news = News(**news)
    # Получаем ссылку на статью
    article_link = 'https://sakhizdat.ru/api/site/matters' + cur_news.path
    # Получаем статью
    article_response = requests.get(article_link).json()
    article = Article(**article_response)
    article.site_link = link
    article.image.image_90 = (link +
                                article.
                                image.
                                image_90[1:])
    article.image.image_800 = (link +
                                article.
                                image.
                                image_800[1:])
    article.image.image_250 = (link +
                                article.
                                image.
                                image_250[1:])
    article.image.image_1600 = (link +
                                article.
                                image.
                                image_1600[1:])
    break

In [23]:
SOURCE_LIST = [
    'Южно-Сахалинск - Газета «Южно-Сахалинск сегодня» - https://sakhizdat.ru/',
    'Корсаковский ГО - Газета «Восход» - https://восход65.рф/',
    'Долинский ГО - Газета «Долинская правда» - https://dolinsk.today/',
    'Ногликский ГО - Газета «Знамя труда» - https://noglgazeta.ru/',
    'Александровск-Сахалинский - Газета «Красное знамя» - https://znamya65.ru/',
    'Курильский - Газета «Красный маяк» - https://iturup.news/',
    'Северо-Курильскиий - Газета «Курильский рыбак» — https://krsevkur.ru/',
    'Южно-Курильский - Газета «На рубеже» — https://kurilnews.ru/',
    'Невельский - Газета «Невельские новости» — https://nevnews.info/',
    'Макаровий - Газета «Новая газета» — https://gazetamakarov.ru/',
    'Смирныховский - Газета «Новая жизнь» — https://alsakh.ru/',
    'Тымовский - Газета «Тымовский вестник» — https://tymnews.ru/',
    'Анивский - Газета «Утро Родины» - https://aniva-utro.ru/',
    'Холмский - Газета «Холмская панорама» - https://kholmsk.info/',
    'Попронайский - Газета «Экспресс» - https://express65.ru/',
    'Томаринский - Газета «Вести Томари» - https://vesti-tomari.ru/',
]

for source in SOURCE_LIST:
    print(source)


Южно-Сахалинск - Газета «Южно-Сахалинск сегодня» - https://sakhizdat.ru/
Корсаковский ГО - Газета «Восход» - https://восход65.рф/
Долинский ГО - Газета «Долинская правда» - https://dolinsk.today/
Ногликский ГО - Газета «Знамя труда» - https://noglgazeta.ru/
Александровск-Сахалинский - Газета «Красное знамя» - https://znamya65.ru/
Курильский - Газета «Красный маяк» - https://iturup.news/
Северо-Курильскиий - Газета «Курильский рыбак» — https://krsevkur.ru/
Южно-Курильский - Газета «На рубеже» — https://kurilnews.ru/
Невельский - Газета «Невельские новости» — https://nevnews.info/
Макаровий - Газета «Новая газета» — https://gazetamakarov.ru/
Смирныховский - Газета «Новая жизнь» — https://alsakh.ru/
Тымовский - Газета «Тымовский вестник» — https://tymnews.ru/
Анивский - Газета «Утро Родины» - https://aniva-utro.ru/
Холмский - Газета «Холмская панорама» - https://kholmsk.info/
Попронайский - Газета «Экспресс» - https://express65.ru/
Томаринский - Газета «Вести Томари» - https://vesti-tomar

In [2]:
# если необходимо сделать сплит по нескольким элементам
# можно реплейсом все привести к единому виду
source.replace('—', '-').split(' - ')

NameError: name 'source' is not defined